In [1]:
%run -i 'src/SPV.py'
from src.data_functions import *
from src.solution_algorithms import *

In [2]:
df = pd.read_csv('data/full_randomized_2017_hourly_normalized_v2.csv')
df.head()

,dataid,localhour,use,gen_per_kW
0,26,2017-01-01 00:00:00,1.788817,0.0
1,26,2017-01-01 01:00:00,1.223917,0.0
2,26,2017-01-01 02:00:00,0.668367,0.0
3,26,2017-01-01 03:00:00,0.478667,0.0
4,26,2017-01-01 04:00:00,0.416283,0.0


In [3]:
dataids = obtain_dataid_2017(df)
dataids_array = np.asarray(dataids)
validdata = len(dataids)
T = len(df[df.dataid==26].use.values)

In [4]:
gen_kw = f_gen_per_kw(df)
load_kw = f_load_kw(df)

1 m$^2$ is 1550 in$^2$

Typical size of solar panel: is 65 in for 39 in = 2535 in$^2$ = 1.6354 m$^2$.

The rated power of a solar panel is typically between 250 to 350W.

To convert we use: gen_per_kw by 0.3 / 1.6354 to get gen_per_m$^2$.

In [5]:
gen_per_m2 = gen_kw*0.3/1.6354

In [6]:
#Investment per m2 is 512.2 $/m2.
#Using an anualized cost with discount rate r=5%, we obtain
CostPVperM2 = 512.2
r = 0.05
years = 20
annuity = CostPVperM2*r/(1-(1+r)**(-years)) #Obtain Annuity 
pi_s = annuity/T #Obtain cost per time step
print('Cost of PV per time step: ' + str(pi_s))

Cost of PV per time step: 0.004695025492240356


In [7]:
firms = validdata #Number of firms
gamma = 0.95 #Factor of NM price
pi_r = 0.18   #Retail Price at 18 cents per kWh.
pi_nm = gamma*pi_r  #Net Metering Price
a_cap_firms = f_cap_firms(gen_per_m2, load_kw) #Cap for firms to not be net producers
a_max_firms = a_cap_firms

In [8]:
investment_standalone = solve_standalone(gen_per_m2, load_kw, dataids, a_max_firms, pi_s, pi_r, pi_nm)

Solving firm 1000 (of 1000) with dataid 990805


In [9]:
print('\n')
print('Investment decisions:')
print(investment_standalone) #Print investment decisions
print('\n')
print('Percentage of investment per firm (with respect to max_cap)')
print(1-(a_max_firms - investment_standalone)/a_max_firms) #Print percentage of investment of available max cap
print('\n Total Investment of PV in standalone case is '+ str(sum(investment_standalone)) + ' in m2 \n')



Investment decisions:
[  0.           0.           0.           0.           0.
  43.22188079  72.81373074   0.           0.           0.
   0.           1.99420927   0.           0.           0.
  42.5193553    0.          59.86670663   0.          11.97763282
   0.           1.7077674    3.687587     0.           0.
  25.87519738  49.01041927  60.03434438   0.          27.37908371
   0.           0.           0.           0.           0.
  45.81730457  38.43321191   0.           6.92813691   0.
  28.41877318  41.64913711   0.           6.54122095   0.
  62.9763341    5.45046099   0.          42.59805109  35.5639713
  38.6611631   15.32375465  34.14931811   0.          44.38703996
  38.74411685   0.           0.           0.           0.
   0.           1.46669105   0.          50.40322522   0.
   0.          35.96039878   0.           0.          26.86995562
   0.           0.           0.           0.           4.09438279
  43.08909255   0.           0.           0.           0.
 

In [10]:
soluc_sharing = solve_sharing_collective(gen_per_m2, load_kw, dataids_array, a_max_firms, pi_s, pi_r)
sol_sharing = firms_investment_sharing(soluc_sharing, a_max_firms, firms)



Iteration: 1
firm removed from S: 91 with dataid 6348
Set of firms that invest:
[148, 186, 546, 751, 407, 153, 777, 919, 392, 939, 712, 882, 906, 187, 493, 146, 97, 488, 182, 602, 173, 144, 5, 542, 156, 123, 534, 982, 90, 145, 594, 658, 839, 264, 721, 807, 730, 279, 849, 586, 726, 497, 958, 57, 613, 749, 350, 377, 411, 185, 29, 65, 81, 180, 582, 302, 447, 262, 690, 828, 758, 754, 678, 272, 274, 95, 519, 285, 576, 703, 840, 794, 345, 619, 785, 851, 476, 298, 791, 408, 806, 369, 626, 741, 484, 940, 536, 6, 366, 92, 996, 160, 164, 409, 647, 925, 944, 796, 830, 725, 645, 991, 652, 237, 503, 38, 116, 314, 815, 845, 921, 579, 444, 983, 504, 309, 451, 896, 967, 817, 217, 677, 448, 872, 178, 642, 24, 618, 827, 396, 155, 660, 885, 912, 235, 970, 585, 165, 943, 232, 553, 167, 571, 172, 689, 655, 867, 657, 735, 963, 631, 701, 486, 457, 59, 862, 945, 740, 696, 771, 811, 731, 70, 732, 236, 545, 318, 876, 787, 934, 965, 210, 539, 833, 421, 17, 598, 713, 466, 813, 744, 544, 593, 623, 31, 286, 995, 

In [14]:
df2 = pd.read_csv('data/LMP_2017_data.csv')
pi_g = f_lmp_prices(df2) #Data is in $/MWh
pi_g = pi_g/1000 # convert to $/kWh

[0.03924907 0.05384699 0.04782042 ... 0.03349952 0.03010856 0.02808794]
8754


In [23]:
pi_g2 = pi_g
soluc_wholesale = solve_wholesale_aggregator(gen_per_m2, load_kw, a_max_firms, pi_s, pi_g2)
print(soluc_wholesale)

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 2000 rows, 1000 columns and 2000 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-03, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+02]
Presolve removed 2000 rows and 1000 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00
optimal value with GUROBI: 46.49627836971733
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [26]:
avg_profit_nopv = utility_profit_no_investment(load_kw, pi_r)

Average profit per time slot without PV investment for utility is 262.51782221582175


In [27]:
avg_profit_standalone = utility_profit_standalone(gen_per_m2, load_kw, investment_standalone, pi_r, pi_nm)

Average profit per time slot for utility (with standalone PV investment) is 212.3057021782694


In [28]:
avg_profit_sharing = utility_profit_sharing(gen_per_m2, load_kw, sol_sharing, pi_r)

Average profit per time slot for utility (with sharing PV investment) is 223.8350651401822


[0.03924907 0.05384699 0.04782042 ... 0.03349952 0.03010856 0.02808794]


[[1.78881667 0.55128333 0.91616667 ... 0.3609     0.21480833 1.55435   ]
 [1.22391667 0.4435     0.4596     ... 0.328325   0.205925   1.80135   ]
 [0.66836667 0.52395    0.35221667 ... 0.28711667 0.238425   1.464125  ]
 ...
 [1.00045    3.74971667 1.02176667 ... 9.952825   0.48616667 2.08993333]
 [1.39793333 3.0895     0.8489322  ... 8.96153333 0.652375   2.57204167]
 [2.53533333 5.07093333 0.42368333 ... 0.4079     2.71175    2.77104167]]
